In [3]:
from owslib.wcs import WebCoverageService
from owslib.util import Authentication
auth = Authentication("luis", "coruja")
wcs = WebCoverageService('http://maps.isric.org/mapserv?map=/map/bdod2.map', version='1.0.0', auth=auth)
print(list(wcs.contents))

ServiceException: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>401 Unauthorized</title>
</head><body>
<h1>Unauthorized</h1>
<p>This server could not verify that you
are authorized to access the document
requested.  Either you supplied the wrong
credentials (e.g., bad password), or your
browser doesn't understand how to supply
the credentials required.</p>
<hr>
<address>Apache/2.4.29 (Ubuntu) Server at maps.isric.org Port 80</address>
</body></html>


In [2]:
print([op.name for op in wcs.operations])

['GetCapabilities', 'DescribeCoverage', 'GetCoverage']


That is a large set of coverages, but it is easy to filter the dictionary. For instance, to get the name of all SoilGrids coverages:

In [3]:
names = [k for k in wcs.contents.keys() if k.startswith("sg250m")]
print(names)

['sg250m:ACDWRB_M_ss_250m', 'sg250m:AWCh1_M_sl1_250m', 'sg250m:AWCh1_M_sl2_250m', 'sg250m:AWCh1_M_sl3_250m', 'sg250m:AWCh1_M_sl4_250m', 'sg250m:AWCh1_M_sl5_250m', 'sg250m:AWCh1_M_sl6_250m', 'sg250m:AWCh1_M_sl7_250m', 'sg250m:AWCh2_M_sl1_250m', 'sg250m:AWCh2_M_sl2_250m', 'sg250m:AWCh2_M_sl3_250m', 'sg250m:AWCh2_M_sl4_250m', 'sg250m:AWCh2_M_sl5_250m', 'sg250m:AWCh2_M_sl6_250m', 'sg250m:AWCh2_M_sl7_250m', 'sg250m:AWCh3_M_sl1_250m', 'sg250m:AWCh3_M_sl2_250m', 'sg250m:AWCh3_M_sl3_250m', 'sg250m:AWCh3_M_sl4_250m', 'sg250m:AWCh3_M_sl5_250m', 'sg250m:AWCh3_M_sl6_250m', 'sg250m:AWCh3_M_sl7_250m', 'sg250m:AWCtS_M_sl1_250m', 'sg250m:AWCtS_M_sl2_250m', 'sg250m:AWCtS_M_sl3_250m', 'sg250m:AWCtS_M_sl4_250m', 'sg250m:AWCtS_M_sl5_250m', 'sg250m:AWCtS_M_sl6_250m', 'sg250m:AWCtS_M_sl7_250m', 'sg250m:BDRICM_M_250m', 'sg250m:BDRLOG_M_250m', 'sg250m:BDTICM_M_250m', 'sg250m:BLDFIE_M_sl1_250m', 'sg250m:BLDFIE_M_sl2_250m', 'sg250m:BLDFIE_M_sl3_250m', 'sg250m:BLDFIE_M_sl4_250m', 'sg250m:BLDFIE_M_sl5_250m', 'sg2

Or to search for all the coverages regarding cation exchange capacity (CEC):

In [4]:
cec_covs = [k for k in wcs.contents.keys() if k.find("CECSOL") != -1]
print(cec_covs)

['sg250m:CECSOL_M_sl1_250m', 'sg250m:CECSOL_M_sl2_250m', 'sg250m:CECSOL_M_sl3_250m', 'sg250m:CECSOL_M_sl4_250m', 'sg250m:CECSOL_M_sl5_250m', 'sg250m:CECSOL_M_sl6_250m', 'sg250m:CECSOL_M_sl7_250m']


These are the SoilGrids predictions for CEC at the seven standard depths.

The details for one of these coverages can be inspected using the identifiers above:

In [6]:
cec0cm = wcs.contents['sg250m:CECSOL_M_sl1_250m']
cec0cm.supportedCRS

[urn:ogc:def:crs:EPSG::4326]

In [7]:
cec0cm.supportedFormats

['ArcGrid', 'GeoTIFF', 'GIF', 'Gtopo30', 'ImageMosaic', 'JPEG', 'PNG', 'TIFF']

A bounding box broadly matching Senegal:

In [8]:
bbox = (-17.1, 12.65, -13.15, 14.95) 

Now fetch the coverage for Senegal:

In [17]:
response = wcs.getCoverage(
    identifier='sg250m:CECSOL_M_sl1_250m', 
    crs='urn:ogc:def:crs:EPSG::4326',
    bbox=bbox, 
    resx=1, resy=1, 
    format='GeoTIFF')

In [18]:
with open('./data/Senegal_CEC0m.tif', 'wb') as file:
    file.write(response.read())

With the data on the client side some regular interaction can be started with a library like resterIO:

In [14]:
import rasterio
CEC = rasterio.open("./data/Senegal_CEC0m.tif", driver="GTiff")
show(CEC, title='Cation Exchange Capacity at 0 metres in Senegal', cmap='gist_ncar')

RasterioIOError: './data/Senegal_CEC0m.tif' not recognized as a supported file format.

Next steps:
https://publicwiki.deltares.nl/display/OET/WCS+primer
https://geoscripting-wur.github.io/PythonRaster/